In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import rosbag

from navlab_turtlebot_common.trajectory import Trajectory
from navlab_turtlebot_planning.linear_planning_model import LinearPlanningModel

%load_ext autoreload
%autoreload 2

In [ ]:
from navlab_turtlebot_common.msg import Control

Control(0, 1)

In [ ]:
lpm_path = os.path.join(os.getcwd(), '..', 'models', 'quadrotor_linear_planning_model.mat')
LPM = LinearPlanningModel(lpm_path)

In [ ]:
v_0 = np.array([0.0, 0.0])
a_0 = np.array([0.0, 0.0])
v_pk = np.array([1.94, -0.06])
k = np.vstack((v_0, a_0, v_pk))
P, V, A = LPM.compute_trajectory(k)

In [ ]:
values = np.stack((P, V, A))
traj = Trajectory(LPM.time, values=values)
traj.compute_twist_controls()

In [ ]:
traj.compute_thetas()

In [ ]:
traj.thetas

In [ ]:
V[:,1]

In [ ]:
# Quiver plot of positions and velocities
import matplotlib.pyplot as plt
plt.quiver(P[:5,0], P[:5,1], V[:5,0], V[:5,1])
# Make axes equal
plt.axis('equal')
plt.show()

In [ ]:
import navlab_turtlebot_common.params as params

DTHETA = 0.5

v_0 = np.array([1.0, 0.0])
theta = np.arctan2(v_0[1], v_0[0])
rands = np.random.rand(params.N_PLAN_MAX, 2)
thetas = theta - DTHETA + 2*DTHETA*rands[:,0]
rs = params.V_MAX_NORM * np.sqrt(rands[:,1])
V_peak = np.vstack((rs*np.cos(thetas), rs*np.sin(thetas))).T

In [ ]:
import matplotlib.pyplot as plt
# Plot v_peak samples
plt.scatter(V_peak[:,0], V_peak[:,1])
# Make axes equal
plt.axis('equal')
plt.show()

Rosbag processing

In [ ]:
datapath = os.path.join('home', 'navlab', 'Data', 'rosbags', 'turtlebot', '4-19-2023')
bagfile = os.path.join(datapath, '2023-04-19-15-16-29.bag')

In [ ]:
bag = rosbag.Bag('2023-04-19-15-16-29.bag', 'r')

In [ ]:
topics = bag.get_type_and_topic_info()[1].keys()

In [ ]:
topics

In [ ]:
topics = ['/turtlebot3/cmd_vel',
          '/turtlebot3/sensing/mocap',
          '/turtlebot3/sensing/mocap_noisy',
          '/turtlebot3/state_est',
          '/turtlebot3/planner/traj']

In [ ]:
x_hist = []
y_hist = []
for topic, msg, t in bag.read_messages(topics=['/turtlebot3/sensing/mocap']):
    x_hist.append(msg.x)
    y_hist.append(msg.y)

In [ ]:
plt.plot(x_hist, y_hist)
plt.axis('equal')
plt.show()

In [ ]:
from navlab_turtlebot_common.utils import unwrap_states

trajs = []
for topic, msg, t in bag.read_messages(topics=['/turtlebot3/planner/traj']):
    trajs.append(unwrap_states(msg.states))

In [ ]:
trajs[0].shape

In [ ]:
fig = plt.figure(figsize=(10,10))
for traj in trajs:
    plt.plot(traj[:,0], traj[:,1])
plt.axis('equal')
plt.show()

In [ ]:
for topic, msg, t in bag.read_messages(topics):
    print(msg)
bag.close()